'''
This creates a data frame which contains Image path BB,PHOS,PHOC representation
'''


In [ ]:
#!python train.py --data coco2.yaml --cfg yolov5s.yaml --weights '' --batch-size 8 --epochs 50

In [ ]:
#! python detect.py --source data/datasets/coco2_test/images --weights ./runs/train/exp6/weights/best.pt --conf 0.25


In [ ]:
#! python detect.py --source data/images --weights ./runs/train/exp3/weights/best.pt --conf 0.25


"""
    BELOW PART PARSES WORD AND ITS COORDINATES
"""

In [ ]:
def mergeCoordinate(d):
    
    '''
    l=d["cord"]
    
    temp=l[0]
    minx,y1=temp[0],temp[1] 
       
    temp=l[-1]
       
    maxx,y2=temp[2],temp[3]    
    
    return minx,y1,maxx,y2
    '''
    
    minX,minY,maxX,maxY=float('inf'),float('inf'),-float('inf'),-float('inf')  
    
    for indx,l in enumerate(d["cord"]):
        
        minX,minY,maxX,maxY=min(minx,l[0]),min(minx,l[1]),max(maxY,l[2]),max(maxY,l[3])
        
#         if indx==0:
#             constX=l[]
            
#         x,y=l[0],l[-1]
        
#         minx,maxY=min(minx,x),max(maxY,y)
    
    return minX,minY,maxX,maxY

        

In [ ]:
#d={'word': 'MOVE', 'cord': [[507, 768, 570, 814], [568, 770, 624, 811], [631, 768, 669, 809], [676, 772, 707, 808], [691, 766, 720, 778]]}
#r=mergeCoordinate(d)
#print("\n\t r:",r)

dumpPath="/home/k/phd/yolov5/data/datasets/forms_1/"

def mergeCoordinate(d):
    
    minX,minY,maxX,maxY=float('inf'),float('inf'),-float('inf'),-float('inf')  
    allCords=[]
    
    #l=d["cord"]
    #l1=l[0]
    #l2=l[-1]
    
    for indx,l in enumerate(d["cord"]):
        
        #print("\n\t l in loop:",l)
        minX,minY,maxX,maxY=min(minX,l[0]),min(minY,l[1]),max(maxX,l[2]),max(maxY,l[3])
        #allCords.append([minX,minY,maxX,maxY])
        #allCords.append(l)
    
    #l=[l1[0],l1[1],l2[-2],l2[-1]]
    l=[minX,minY,maxX,maxY]
    allCords.append(l)
    return allCords#minX,minY,maxX,maxY

def show(img,nm):
    #img.save("./temp/"+nm+".png")

    img=img.resize((1000,800))
    img.show()

#df=["image_name","class","width","height","x1","y1","x2","y2","org_x1","org_y1","org_x2","org_y2"]

def fill(indx,img,nm,fileName,df,x1,y1,x2,y2,currTextVal):
    
    w,h=img.size
    df.loc[indx,"image_name"]=nm
    df.loc[indx,"class"]=1
    df.loc[indx,"width"]=w
    df.loc[indx,"height"]=h
    df.loc[indx,"x"]=round((x1+x2)/(2*w),2) 
    df.loc[indx,"y"]=round((y1+y2)/(2*h),2)
    df.loc[indx,"w"]=round(abs(x2-x1)/w,2)
    df.loc[indx,"h"]=round(abs(y2-y1)/h,2)
    df.loc[indx,"org_x1"]=x1
    df.loc[indx,"org_y1"]=y1
    df.loc[indx,"org_x2"]=x2
    df.loc[indx,"org_y2"]=y2
    df.loc[indx,"text"]=currTextVal
    
    xx=str(round((x1+x2)/(2*w),2))
    yy=str(round((y1+y2)/(2*h),2))
    ww=str(round(abs(x2-x1)/w,2))
    hh=str(round(abs(y2-y1)/h,2))
    
    text="0"+" "+xx+" "+yy+" "+ww+" "+hh
    
    with open (dumpPath+fileName,'a') as f: 
        f.write (text+'\n')
    
    return df


'''
    below part parse forms and records coordinate and text 
    
'''

In [ ]:
#[elem.tag for elem in root.iter()]
from collections import defaultdict
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import os
import pandas as pd
import sys
d={}


#imgPath="/home/k/phd/yolov5/data/datasets/formsA-D/a01-000u.png"
imgPath="/home/k/phd/yolov5/data/datasets/forms/"
path="/home/k/phd/yolov5/data/xml//"
#files=["a01-000u.xml","a01-000x.xml"]
allImages=os.listdir(imgPath)
files=os.listdir(path)
df=pd.DataFrame(columns=["image_name","class","width","height","org_x1","org_y1","org_x2","org_y2","text"])
lineNo=0
expCount=0
for indx,nm in enumerate(files):
    
    
    '''
    if indx not in [20,22,32,33,41,46,47,62]:
        continue
        
    if indx>62:
        break
    ''' 
    imgName=nm.split(".xml")[0]+".png"
    if imgName not in allImages:
        continue
    
    if indx%100==0:
        print("\n\t indx:",indx,"\t total:",len(files),"\t\t nm:",nm,"\t unique files:",len(df.image_name.unique()))
        
#     if len(df.image_name.unique())>200:
#         break

    doc = ET.parse(path+nm)
    root = doc.getroot()
    fn=nm.split(".xml")[0]+".txt"

    img=Image.open(imgPath+imgName)
    #img=img.convert('RGB')
    #print("\n\t 1.shape:",img.size)     
    #show(img,"nm")
    #input("check")
#     continue
    img1 = ImageDraw.Draw(img) 

    for node in root.iter('word'):
        #print("\n\t attrib:",node.attrib["text"])
        currTextVal=node.attrib["text"]
        #print("\n\t ",i)
        prevWord=None
        words=defaultdict(list)

        for ele in node.iter():
            #print("\n\t ele:",ele.tag)
            if ele.tag=="word":

                try:
                    if "word" in d.keys():
                        prevWord=d["word"]
                        #print("\n\t prevWord=",d["word"],"\t word:",ele.attrib["text"])
                        
                        #print("\n\t d:",d)
                        allCords=mergeCoordinate(d)
                        #print("\n\t allCords=",allCords)
                        cord=allCords[0]
                        #for cord in allCords:
                        #print("\n\t\t\t cord:",cord)
                        x1,y1,x2,y2=int(cord[0]),int(cord[1]),int(cord[2]),int(cord[3])
                        #area=abs(x2-x1)*abs(y2-y1)
                        #print("\n\t area:",area)
                        #if 1:#abs(x2-x1)*abs(y2-y1)<300000:
                        #img1.line((x1,(y1+y2)/2,x2,(y1+y2)/2),fill="black",width=10)
                        
                        df=fill(lineNo,img,imgName,fn,df,x1,y1,x2,y2,currTextVal)
                        lineNo+=1
                        #print("\n\t img=",img.shape)
                        #img1.rectangle(((x1,y1), (x2,y2)),outline = "red",width=10)
                        #show(img,nm)
                        #input(3)

                except Exception as e:
                    expCount+=1
                    exc_type, exc_obj, exc_tb = sys.exc_info()
                    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                    #print(exc_type, fname, exc_tb.tb_lineno)
                    #print("\n\t  d exception:",e,"\t line number:",exc_tb.tb_lineno,"\t ",indx)
                    #print("\n\t cord:",cord,"\t exp:",expCount,d)
                    #input("check")

                d={}
                d["word"]=ele.attrib["text"]
                d["cord"]=[]
            elif ele.tag=="cmp":
                #print("\n\t\t current text:",d["word"])
                #print("\n\t\t current co-ordinate:",ele.attrib)
                #print("\n\t\t x:",ele.attrib['x'],"\t y:",ele.attrib['y'])
                x1,y1=int(ele.attrib['x']),int(ele.attrib['y'])
                x2,y2=x1+int(ele.attrib['width']),y1+int(ele.attrib['height'])
                #img1.line((x1,y1,x2,y2),fill="green",width=10)                
                #img1.rectangle(((x1,y1), (x2,y2)), fill="green")

                d["cord"].append([x1,y1,x2,y2])

                allCords=mergeCoordinate(d)
                cord=allCords[0]
                x1,y1,x2,y2=cord[0],cord[1],cord[2],cord[3]
                area=abs(x2-x1)*abs(y2-y1)
                #img1.rectangle(((x1,y1), (x2,y2)),outline = "red",width=3)
                #d={}
                
        if indx%5==0:
            df.to_csv("./data/data3.csv",index=False)

    df.to_csv("./data/data3.csv",index=False)
    #print("\n\t uunique files:",len(df.image_name.unique()))    
    #img.save("./temp/"+nm+".png")
    #img=img.resize((500,400))
    #show(img,imgName)
    #input("press")


In [ ]:
df.columns

'''
    now add phoc and phos label
'''

In [ ]:
from phos_label_generator import gen_label
from phoc_label_generator import gen_phoc_label

#generate_phoc_vector("A")

In [ ]:
train_word_phos_label = gen_label(list(set(df['text'])))
print("\n\t len:",len(train_word_phos_label))

train_word_phoc_label = gen_phoc_label(list(set(df['text'])))
print("\n\t len:",len(train_word_phoc_label))


In [ ]:
import re

len(list(set(df['text'])))
word=list(set(df['text']))


regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string

tempPhos=[0]*165
tempPhoc=[0.0]*604
#output['Sold Count'] = output['Sold Count'].astype(object)
df["phos"]=df["phos"].astype(object)

for indx,row in df.iterrows():
    
#     if indx>100:
#         break
    
    try:
        #print("\n\t w1:",w)
        w=row["text"]
        w=regex.sub('', w)
        #print("\n\t w2:",w)
        if len(w):
            phos_label = gen_label([w])
            phoc_label = gen_phoc_label([w])
            #print("\n\t train_word_phos_label=",train_word_phos_label)
            #print("\n\t keys:",len(phos_label[w]))
            #print("\n\t type:",type(phos_label[w]))
            #print("\n\t phos_label[w]=",phos_label[w])
            #df.loc[indx,"phoc"]=str(phos_label[w]) gen_phoc_label
            
            #print("\n\t phos_label =",len(phos_label[w].tolist()))
            #print("\n\t phoc_label =",len(phoc_label[w]))

            df.at[indx,"phoc"]=phoc_label[w]
            df.at[indx,"phos"]=phos_label[w]#.tolist()

        else:
            df.at[indx,"phoc"]=tempPhoc
            df.at[indx,"phos"]=tempPhos

        #input("check!!")
        
        
    except Exception as e:
        print("\n\t w:",w,"\t indx:",indx,"\t e:",e)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\n\t line number:", exc_tb.tb_lineno)
        input("check!!!")
df.to_csv("./data/data3_phosc.csv")
#train_word_phos_label = gen_label(list(set(df['text'])))
#train_word_phos_label = gen_label(word[5])

In [ ]:
import ast
#df.dtypes
phos=df.loc[0,"phoc"]
#phos
#phos[7]
#len(phos)

#phos_label = gen_label([w])
#type(phos_label[w].tolist())
#type(phoc_label[w])
#phos_label[w]#.tolist()

In [ ]:
#!conda info

In [ ]:
!conda install Pillow --y

In [ ]:
import os
os.getcwd()

In [ ]:
import os
from PIL import Image
path="/home/k/phd/yolov5/runs/detect/exp7/"
path1="/home/k/phd/yolov5/runs/detect/compress/"

imgList=os.listdir(path)

for indx,nm in enumerate(imgList):
    
    img=Image.open(path+nm)
    img=img.resize((500,400))
    
    img.save(path1+nm)



In [ ]:
'''
    text file creation for YOLO
'''

In [ ]:
# Python program to illustrate
# Append vs write mode
file1 = open("myfile.txt", "w")
L = ["This is Delhi \n", "This is Paris \n", "This is London"]
file1.writelines(L)
file1.close()

In [ ]:

import pandas as pd
data=pd.read_csv("/home/k/phd/yolov5/data/data3.csv")
data.shape
print(data.columns)

In [ ]:
os.getcwd()
data.dtypes

In [ ]:
#for file in ['train','test']:
import numpy as np
import os
'''
allImages=os.listdir("/home/k/phd/yolov5/data/datasets/forms/")
images_num=len(os.listdir("/home/k/phd/yolov5/data/datasets/forms/"))
images_path = "/home/k/phd/yolov5/data/datasets/forms/"#os.getcwd()+f"/mnist_{file}"
labels_txt = os.getcwd()+"/"+"mnist_train.txt"
labels_txt2 = os.getcwd()+"/"+"mnist_test.txt"
'''

allImages=os.listdir("/home/k/phd/yolov5/data/datasets/forms/")
images_num=len(os.listdir("/home/k/phd/yolov5/data/datasets/forms/"))
images_path = "/home/k/phd/yolov5/data/datasets/forms/"#os.getcwd()+f"/mnist_{file}"
labels_txt = os.getcwd()+"/"+"mnist_train.txt"
labels_txt2 = os.getcwd()+"/"+"mnist_test.txt"



lastName=""
curLine=""
prevLine=""
unqImg=[]
exceptionCount=0

for indx,info in df.iterrows():
    try:
        if indx%10==0:
            print("\n\t unq images:",len(unqImg))

        if len(unqImg)<100:

            with open(labels_txt, "a") as wf:
                image_path =images_path+info["image_name"]

                if indx%100==0:
                    print("\n\t path:",image_path)

                if info["image_name"]!=lastName and info["image_name"] not in unqImg:
                    curLine=image_path

                    if len(prevLine):
                        #print("\n\t prevLine:",prevLine)
                        #wf.write( image_path+ "\n")
                        wf.write(prevLine+"\n")

                else:
                    xmin,ymin=str(int(info["org_x1"])),str(int(info["org_y1"]))
                    xmax,ymax=str(int(info["org_x2"])),str(int(info["org_y2"]))
                    curLine += ' ' + ','.join([xmin, ymin, xmax, ymax, str(0)])
                    prevLine=curLine

                lastName=info["image_name"]

        elif 100<len(unqImg)<200:

            with open(labels_txt2, "a") as wf:
                image_path =images_path+info["image_name"]

                if indx%100==0:
                    print("\n\t path:",image_path)

                if info["image_name"]!=lastName:
                    curLine=image_path

                    if len(prevLine):
                        #print("\n\t prevLine:",prevLine)
                        #wf.write( image_path+ "\n")
                        wf.write(prevLine+"\n")

                else:
                    xmin,ymin=str(int(info["org_x1"])),str(int(info["org_y1"]))
                    xmax,ymax=str(int(info["org_x2"])),str(int(info["org_y2"]))
                    curLine += ' ' + ','.join([xmin, ymin, xmax, ymax, str(0)])
                    prevLine=curLine

                lastName=info["image_name"]
        elif 200<len(unqImg):
            break
            
        if info["image_name"] not in unqImg:
            unqImg.append(info["image_name"])

    except Exception as e:
        exceptionCount+=1
        print("\n\t exception index:",indx,"\t count:",exceptionCount)


In [2]:
'''
    below part convert dataframe to json 
'''
import pandas as pd
df=pd.read_csv("/home/k/phd/TensorFlow-2.x-YOLOv3/data/data3_phosc.csv")
print("\n\t df:",df.shape)
df.columns


	 df: (115187, 16)


Index(['Unnamed: 0', 'image_name', 'class', 'width', 'height', 'org_x1',
       'org_y1', 'org_x2', 'org_y2', 'text', 'x', 'y', 'w', 'h', 'phoc',
       'phos'],
      dtype='object')

In [6]:
df.dtypes

Unnamed: 0      int64
image_name     object
class           int64
width           int64
height          int64
org_x1          int64
org_y1          int64
org_x2          int64
org_y2          int64
text           object
x             float64
y             float64
w             float64
h             float64
phoc           object
phos           object
dtype: object

In [7]:
import collections
import json
import numpy as np
d=collections.defaultdict(dict)
tempDir={}
tempCord={}
tempText={}

for indx,info in df.iterrows():
    
    tempDir2={}
    #print("\n\t name:",info["image_name"])
    temp=[info['org_x1'],info['org_y1'],info['org_x2'],info['org_y2']]
    '''
    tempDir2["text"]=info['text']
    '''
    tempDir2["phoc"]=info['phoc']
    tempDir2["phos"]=info['phos']
    #tempDir[str(temp)]=tempDir2

    tempCord[str(temp)]=info['text']
    tempText[info['text']]=tempDir2
    
with open("./data/tempCord.json", "w") as outfile:
    json.dump(tempCord, outfile)

with open("./data/tempText.json", "w") as outfile:
    json.dump(tempText, outfile)




In [8]:


f = open("./data/tempCord.json")
tempCord1= json.load(f)

f = open("./data/tempText.json")
tempText1 = json.load(f)



'''
with open("./data/dataframe.json", "w") as outfile:
    json.dump(d, outfile)
'''


'\nwith open("./data/dataframe.json", "w") as outfile:\n    json.dump(d, outfile)\n'

In [13]:
tempCord1

{'[361, 587, 480, 652]': 'it',
 '[507, 600, 550, 648]': 'his',
 '[603, 603, 689, 652]': 'imagination',
 '[768, 593, 1120, 686]': 'or',
 '[1164, 620, 1209, 648]': 'was',
 '[1250, 620, 1370, 646]': 'the',
 '[1408, 595, 1503, 639]': 'panel',
 '[1531, 599, 1676, 643]': 'slightly',
 '[1714, 579, 1928, 678]': 'lop-sided',
 '[1957, 584, 2179, 646]': '?',
 '[2188, 572, 2210, 636]': 'Leaping',
 '[360, 771, 597, 857]': 'forward',
 '[610, 775, 860, 835]': 'with',
 '[895, 780, 1018, 827]': 'a',
 '[1049, 794, 1094, 822]': 'choked',
 '[1130, 763, 1319, 833]': 'sound',
 '[1366, 775, 1534, 826]': 'he',
 '[1579, 774, 1655, 823]': 'grasped',
 '[1696, 772, 1899, 849]': 'the',
 '[1928, 774, 2050, 816]': 'oblong',
 '[2084, 763, 2252, 843]': 'panel',
 '[352, 958, 500, 1024]': 'and',
 '[571, 946, 675, 1010]': 'pulled',
 '[743, 943, 896, 1020]': 'it',
 '[925, 948, 957, 1001]': 'out',
 '[989, 954, 1076, 1008]': '.',
 '[1087, 994, 1098, 999]': 'The',
 '[1154, 959, 1300, 1001]': 'black',
 '[1352, 942, 1539, 997]

In [2]:
import json    
f = open("./data/dataframe.json")
 
data = json.load(f)

data['l04-087.png']#['[361, 587, 480, 652]']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)